In [12]:
import pandas as pd
import numpy as np
import os
import re

In [13]:
os.getcwd()

'/Users/lilimatic/chris/chris/_output/hedges/pnl/20210228_021905'

In [14]:
os.chdir('/Users/lilimatic/chris/chris/_output/hedges/pnl/20210228_021905')

In [15]:
DIR = "20210228_021905"
PNL = "PNL__KDE__DeltaGammaHedge__BULLISH__3000__30.csv"

In [16]:
prices = pd.read_csv("/Users/lilimatic/chris/chris/_output/hedges/pnl/20210228_021905/__prices.csv")
prices = dict(prices[["label", "price"]].to_numpy())

In [17]:
rows = []

stats = ['mean', 'std', 'skew', 'kurt','1%','25%','50%','75%','99%']

for fname in [fname for fname in os.listdir(".") if fname.endswith('.csv') and fname.startswith('PNL')]:
    paths, model, hedge, period, strike, maturity = re.match("PNL__(\w+)__(\w+)__(\w+)__(\w+)__(\w+)__(\w+)__100000.csv", fname).groups()
    df = pd.read_csv(f"{fname}", header=None)
    label = re.match(".*_(\w+__\d+__\d+)__\d+.csv$", fname).groups()[0]
    price = prices[label]
    quantile_low = df.quantile(0.01)
    quantile = df.quantile(.99)
    skew = df.skew()
    kurt = df.kurtosis()
    df = df.describe()
    df = df.T
    df['skew'] = skew
    df['kurt'] = kurt
    df['1%'] = quantile_low
    df['99%'] = quantile
    df = np.round(df/price, 3)
    del df['count']
    del df['min']
    del df['max']
    df = df[stats]
    rows += [[paths, model, hedge, period, strike, maturity, *df.to_numpy()[0]]]
    
columns = ['paths', 'model', 'hedge', 'period', 'strike', 'maturity', *stats]
df = pd.DataFrame(rows, columns=columns)

df = df[df['maturity'] == '90']
df = df[df['strike'].isin(['3000', '7000', '9000'])]
df

,paths,model,hedge,period,strike,maturity,mean,std,skew,kurt,1%,25%,50%,75%,99%
1,KDE,MERTON,DeltaVegaHedge,BULLISH,3000,90,0.053,0.391,0.001,0.004,-0.888,-0.149,0.044,0.227,1.272
2,SVCJ,BLACK_SCHOLES,DeltaGammaHedge,BULLISH,3000,90,0.020,0.121,-0.000,0.016,-0.336,-0.025,0.017,0.072,0.317
4,KDE,SVCJ,DeltaVegaHedge,CALM,7000,90,0.043,0.083,0.000,0.001,-0.164,-0.003,0.044,0.086,0.260
16,SVCJ,CGMY,DeltaHedge,CALM,7000,90,-0.024,0.402,-0.001,0.008,-1.306,-0.144,0.065,0.171,0.825
17,KDE,HESTON,MinimumVarianceHedge,COVID,9000,90,-0.219,0.368,-0.000,0.001,-1.216,-0.438,-0.193,0.024,0.587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848,KDE,VARIANCE_GAMMA,DeltaHedge,BULLISH,3000,90,0.007,0.392,0.001,0.004,-0.942,-0.195,0.001,0.182,1.222
850,KDE,VARIANCE_GAMMA,DeltaGammaHedge,CALM,9000,90,-0.030,0.148,0.000,0.004,-0.465,-0.082,-0.037,0.022,0.422
853,KDE,SVCJ,MinimumVarianceHedge,CALM,9000,90,0.616,0.218,0.001,0.000,0.218,0.455,0.596,0.755,1.203
862,KDE,MERTON,MinimumVarianceHedge,CALM,9000,90,0.578,0.642,-0.000,-0.001,-0.743,0.073,0.603,1.085,1.867


In [18]:
hs = {
    'DeltaHedge': 0,
    'DeltaGammaHedge': 1,
    'DeltaVegaHedge': 2,
    'MinimumVarianceHedge': 3,
}

ms = {
    'BLACK_SCHOLES': 0,
    'HESTON': 1,
    'VARIANCE_GAMMA': 2,
    'SVJ': 3,
    'SVCJ': 4,
    'CGMY': 5,
    'MERTON': 6,
}

hn = {
    'DeltaHedge': "\Delta ",
    'DeltaGammaHedge': "\Delta-\Gamma ",
    'DeltaVegaHedge': "\Delta-\mathcal V ",
    'MinimumVarianceHedge': "\textbf{MV} ",
}

mn = {
    'BLACK_SCHOLES': "BS",
    'HESTON': "SV",
    'VARIANCE_GAMMA': "VG",
    'SVJ': 'SVJ',
    'SVCJ': 'SVCJ',
    'CGMY': 'CGMY',
    'MERTON': 'Merton',
}

for (paths, period), odf in df.groupby(['paths', 'period']):
    print("# ", paths, period, "\n\n")
    odf['hs'] = odf.apply(lambda x: hs[x['hedge']], axis=1)
    odf['ms'] = odf.apply(lambda x: ms[x['model']], axis=1)
    odf['name'] = odf.apply(lambda x: f"${hn[x['hedge']]}_{{{mn[x['model']]}}}$", axis=1)
    odf = np.round(odf, 3)
    odf = odf.sort_values(['hs', 'ms'])[['name', *stats]]
    odf.columns = ['name', 'mean', 'std', 'skew', 'kurt', '1\%', '25\%', '50\%', '75\%', '99\%']
    caption = f"Summary statistics of {period} {paths} hedges"
    label = f"tab:{paths}_{period}"
    print(f"\\begin{{table}}\n\centering\n{odf.to_latex(index=False, escape=False)}\n\\caption{{{caption}}}\n\\label{{{label}}}\n\\end{{table}}")
    print('\n==================================================================\n\n')

#  KDE BULLISH 


\begin{table}
\centering
\begin{tabular}{lrrrrrrrrr}
\toprule
                          name &   mean &    std &   skew &   kurt &    1\% &   25\% &   50\% &   75\% &   99\% \\
\midrule
                $\Delta _{BS}$ &  0.008 &  0.392 &  0.001 &  0.004 & -0.940 & -0.194 &  0.001 &  0.183 &  1.223 \\
                $\Delta _{SV}$ &  0.009 &  0.392 &  0.001 &  0.004 & -0.940 & -0.194 &  0.002 &  0.183 &  1.227 \\
                $\Delta _{VG}$ &  0.007 &  0.392 &  0.001 &  0.004 & -0.942 & -0.195 &  0.001 &  0.182 &  1.222 \\
               $\Delta _{SVJ}$ &  0.008 &  0.392 &  0.001 &  0.004 & -0.940 & -0.195 &  0.001 &  0.183 &  1.225 \\
              $\Delta _{SVCJ}$ &  0.009 &  0.392 &  0.001 &  0.004 & -0.938 & -0.194 &  0.002 &  0.184 &  1.228 \\
              $\Delta _{CGMY}$ &  0.008 &  0.392 &  0.001 &  0.004 & -0.942 & -0.195 &  0.001 &  0.182 &  1.222 \\
            $\Delta _{Merton}$ &  0.009 &  0.392 &  0.001 &  0.004 & -0.937 & -0.194 &  0.002 &  0.184 &  

In [19]:
for fname in [fname for fname in os.listdir(".") if fname.endswith('4000__30__100000.csv') and fname.startswith('PNL__SVCJ__BLACK_SCHOLES')]:
    df = pd.read_csv(f"{fname}", header=None)
    label = re.match(".*_(\w+__\d+__\d+)__\d+.csv$", fname).groups()[0]
    price = prices[label]
    print(price)
    quantile_low = df.quantile(0.01)
    quantile = df.quantile(.99)
    skew = df.skew()
    kurt = df.kurtosis()
    df = df.describe()
    df = df.T
    df['skew'] = skew
    df['kurt'] = kurt
    df['1 %'] = quantile_low
    df['99 %'] = quantile
    df = np.round(df/price, 4)
    del df['count']
    del df['min']
    del df['max']
    df = df[['mean', 'std', 'skew', 'kurt','1 %','25%','50%','75%','99 %']]
    print(fname)
    display(df)
    print("\n\n")
    print(df.to_latex())
    print("\n\n")

201.9261905072699
PNL__SVCJ__BLACK_SCHOLES__DeltaVegaHedge__BULLISH__4000__30__100000.csv


,mean,std,skew,kurt,1 %,25%,50%,75%,99 %
0,0.0277,0.5132,0.0186,0.6715,-1.418,-0.0662,0.0803,0.2165,1.0





\begin{tabular}{lrrrrrrrrr}
\toprule
{} &    mean &     std &    skew &    kurt &    1 \% &     25\% &     50\% &     75\% &  99 \% \\
\midrule
0 &  0.0277 &  0.5132 &  0.0186 &  0.6715 & -1.418 & -0.0662 &  0.0803 &  0.2165 &   1.0 \\
\bottomrule
\end{tabular}




201.9261905072699
PNL__SVCJ__BLACK_SCHOLES__DeltaHedge__BULLISH__4000__30__100000.csv


,mean,std,skew,kurt,1 %,25%,50%,75%,99 %
0,0.103,0.567,-0.0122,0.2019,-1.8879,-0.0542,0.1501,0.3695,1.0928





\begin{tabular}{lrrrrrrrrr}
\toprule
{} &   mean &    std &    skew &    kurt &     1 \% &     25\% &     50\% &     75\% &    99 \% \\
\midrule
0 &  0.103 &  0.567 & -0.0122 &  0.2019 & -1.8879 & -0.0542 &  0.1501 &  0.3695 &  1.0928 \\
\bottomrule
\end{tabular}




201.9261905072699
PNL__SVCJ__BLACK_SCHOLES__DeltaGammaHedge__BULLISH__4000__30__100000.csv


,mean,std,skew,kurt,1 %,25%,50%,75%,99 %
0,0.0279,0.5143,0.0189,0.6854,-1.4177,-0.066,0.0808,0.2167,1.0001





\begin{tabular}{lrrrrrrrrr}
\toprule
{} &    mean &     std &    skew &    kurt &     1 \% &    25\% &     50\% &     75\% &    99 \% \\
\midrule
0 &  0.0279 &  0.5143 &  0.0189 &  0.6854 & -1.4177 & -0.066 &  0.0808 &  0.2167 &  1.0001 \\
\bottomrule
\end{tabular}






In [20]:
os.getcwd()

'/Users/lilimatic/chris/chris/_output/hedges/pnl/20210228_021905'

In [21]:
df

,mean,std,skew,kurt,1 %,25%,50%,75%,99 %
0,0.0279,0.5143,0.0189,0.6854,-1.4177,-0.066,0.0808,0.2167,1.0001
